In [ ]:
import glob
import os

import dask

import dask as da
import dask.dataframe as dd
import datashader as ds
import holoviews as hv
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize


from bokeh.models.ranges import DataRange1d, Range1d

import pandas as pd

In [ ]:
hv.extension('bokeh')

In [ ]:
tract = 4850
patch = '34'

data_dir = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/summary'

datafile_tract_patch_pattern = os.path.join(data_dir, 'dpdd_object_tract_%d_patch_%s.parquet' % (tract, patch))
datafile_tract_pattern = os.path.join(data_dir, 'dpdd_object_tract_%d.parquet' % tract)
datafile_all_pattern = os.path.join(data_dir, 'dpdd_object_tract_????.parquet')

datafile_tract = os.path.join(data_dir, 'dpdd_object_tract_%d.parquet' % tract)
datafile_all = os.path.join(data_dir, 'dpdd_object.parquet')

In [ ]:
#dask read_parquet doesn't glob the same way glob does
# So we global here to get our file list and then pass that to dask.read_parquet
datafile_all_list = glob.glob(datafile_all_pattern)

In [ ]:
datafile = datafile_tract
# datafile = datafile_all_list

df = dd.read_parquet(datafile, columns=['ra', 'dec', 'mag_g', 'mag_r', 'mag_i', 'magerr_g', 'magerr_r', 'magerr_i'])

In [ ]:
print(df.columns)

In [ ]:
# Clean
snr_magerr_threshold = 0.3  # mag
df = df[(df.magerr_g < snr_magerr_threshold) & (df.magerr_r < snr_magerr_threshold) & (df.magerr_i < snr_magerr_threshold)]

In [ ]:
df['g-r'] = df['mag_g'] - df['mag_r']
df['r-i'] = df['mag_r'] - df['mag_i']

In [ ]:
points_ra_dec = hv.Points(df, kdims=['ra', 'dec'], vdims=['g-r'])
ra_dec = datashade(points_ra_dec, aggregator=ds.mean('g-r'))

In [ ]:
points_mag_magerr = hv.Points(df, kdims=[hv.Dimension('mag_g', soft_range=(14, 28)),
                                         'magerr_g'])
mag_magerr = datashade(points_mag_magerr)

In [ ]:
points_color_mag = hv.Points(df, kdims=[hv.Dimension('g-r', soft_range=(-2, 3)),
                                        hv.Dimension('mag_g', soft_range=(28, 14))])
color_mag = datashade(points_color_mag)

In [ ]:
points_color_color = hv.Points(df, kdims=[hv.Dimension('g-r', soft_range=(-2, 3)),
                                          hv.Dimension('r-i', soft_range=(-2, 3))])
# color_color = datashade(points_color_color.hist(dimension=['g-r', 'r-i']))
color_color = datashade(points_color_color)

In [ ]:
ra_dec + color_mag + color_color

In [ ]:
%%opts QuadMesh [tools=['hover']] (alpha=0 hover_alpha=0.2)
from holoviews.streams import RangeXY

dynamic_mag_magerr = datashade(points_mag_magerr, width=400, height=400) * \
    hv.util.Dynamic(rasterize(points_mag_magerr, width=10, height=10, streams=[RangeXY]), operation=hv.QuadMesh)

dynamic_color_mag = datashade(points_color_mag, width=400, height=400) * \
    hv.util.Dynamic(rasterize(points_color_mag, width=10, height=10, streams=[RangeXY]), operation=hv.QuadMesh)

dynamic_color_color = datashade(points_color_color, width=400, height=400) * \
    hv.util.Dynamic(rasterize(points_color_color, width=10, height=10, streams=[RangeXY]), operation=hv.QuadMesh)

In [ ]:
dynamic_mag_magerr + dynamic_color_mag + dynamic_color_color

In [ ]:
len(df)

In [ ]:
hv.help(color_mag)